In [2]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

wiki.shape, mrc.shape

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


((60613, 8), (4192, 7))

In [3]:
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    'q1',
    'q2'
]

# No need to add instruction for retrieval documents
documents = [
    "d1",
    "d2",
    "d3"
]
input_texts = queries + documents
input_texts

['q1', 'q2', 'd1', 'd2', 'd3']

In [4]:
from sentence_transformers import SentenceTransformer

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, '한글이야'),
    get_detailed_instruct(task, 'english girl, guideline women ages')
]
# No need to add instruction for retrieval documents
documents = [
    "한글이야",
    "english girl",
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅",
    "응 이거 아니야"
]
input_texts = queries + documents

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

embeddings = model.encode(input_texts, convert_to_tensor=True, normalize_embeddings=True)
print(embeddings.shape)
scores = (embeddings[:len(queries)] @ embeddings[len(queries):].T) * 100
print(scores.tolist())
# [[91.92853546142578, 67.5802993774414], [70.38143157958984, 92.13307189941406]]


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacity of 31.74 GiB of which 476.38 MiB is free. Process 3891637 has 30.97 GiB memory in use. Process 3909762 has 306.00 MiB memory in use. Of the allocated memory 0 bytes is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
import random
random.seed(42)

query_n = 50 # query number
wiki_n = 100 # wiki number
batch_size = 5  # 한번에 처리할 문서 수 설정
top_k = 5

sampled_indices = random.sample(range(len(mrc)), query_n)
queries = [mrc['question'].iloc[idx] for idx in sampled_indices]
correct_contexts = [mrc['context'].iloc[idx] for idx in sampled_indices]
wiki_list = random.sample(wiki['text'].tolist(), wiki_n - query_n) + correct_contexts
print(sampled_indices)
print(queries[0])
print(correct_contexts[0])
print(f'queries len: {len(queries)}')
print(f'wiki_list len: {len(wiki_list)}')

for i in range(len(queries)):
    print(f'Query: {queries[i]}')
    print(f'Correct context: {correct_contexts[i]}')
    print()

[912, 204, 2253, 2006, 1828, 1143, 839, 712, 3456, 260, 244, 767, 1791, 1905, 4139, 217, 1628, 3436, 1805, 3679, 2278, 53, 1307, 3462, 2787, 2276, 1273, 1763, 2757, 837, 759, 3112, 792, 2940, 2817, 2166, 355, 3763, 1022, 3100, 645, 2401, 2962, 1575, 569, 375, 1866, 2370, 653, 1907]
이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?
항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리학자이자 보성군의 갑부가 되었으며 참판을 역임하기도 했다. 그의 증대고모할머니 성주이씨는 1860년대 초 동복군수를 지낸 서광효에게 출가했는데, 서재필과 서재창이 그들의 아들이었다. 갑신정변 실패 후 그의 집안은 연좌제로 사형당하지 않을까 오랫동안 전전긍긍했다 한다.\n\n일제 강점기에 그의 집안은 독립운동에 투신하였다. 그는 경성법학전문학교(서울대학교 법과 대학의 전신)을 다녔다.\n\n경성법전을 졸업한 용순은 낙향, 학숙을 열며 고향을 일구었다. \n\n그의 3형제 모두 끝까지 창씨개명을 거부했다 1947년 서재필은 귀국하자마자 외종손 용순을 만났다. 서재필은 눈물을 흘리며 "과거에 나로 인하여 외가에 많은 피해를 주었다. 고향에 돌아가 가세를 유지하길 바란다"고 했다 아버지 이일은 미국에 있던 내종숙 서재필과 서신을 주고받았다. 그는 이 편지서신들을 보관하고 있었다. 그러나 아버지 이일과 서재필 주고 받았던 서신은 한국전쟁때 집과 함께 불타 없어졌다고 한다\n\n1947년 서재필이 미국으로 돌아갈 때 용순에게 '제발 정치에 참여하지 말고 고향에 돌아가 수신제가(修身齊家)하고 가업에 충실하라'하고 당부하였다. 이후 이용

In [1]:
import json
import pandas as pd
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
import numpy as np
import random

random.seed(42)

'''data'''
data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T
dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])


'''inference'''
model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")

query_n = 50 # query number
wiki_n = 100 # wiki number
batch_size = 5  # 한번에 처리할 문서 수 설정
model.max_seq_length = 2048
top_k = 5

# wiki = random wiki + correct answer included
sampled_indices = random.sample(range(len(mrc)), query_n)
queries = [mrc['question'].iloc[idx] for idx in sampled_indices]
correct_contexts = [mrc['context'].iloc[idx] for idx in sampled_indices]
wiki_list = random.sample(wiki['text'].tolist(), wiki_n - query_n) + correct_contexts
random.shuffle(wiki_list)

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.41s/it]


In [7]:
queries = queries[:1]
wiki_list = wiki_list[:10]

for i in range(len(queries)):
    query_embedding = model.encode(queries[i], prompt_name="web_search_query")

    # 배치로 document embedding을 처리
    all_scores = []
    for start_idx in tqdm(range(0, len(wiki_list), batch_size), desc=f"Processing query {i+1}/{len(queries)}"):
        print(f'start idx: {start_idx}')
        print(f'batch size: {batch_size}')
        print(f'wiki list len: {len(wiki_list)}')
        batch_wiki_list = wiki_list[start_idx:start_idx + batch_size]
        document_embeddings = model.encode(batch_wiki_list)
        scores = (query_embedding @ document_embeddings.T) * 100
        print(scores.shape)
        all_scores.append(scores)
    print(all_scores)

    # 점수 결합
    all_scores = np.concatenate(all_scores, axis=-1)
    print(all_scores)
    
    # 상위 top_k 문서 찾기
    top_indices = np.argsort(all_scores)[::-1][:top_k]
    print(top_indices)
    
    correct = False
    for idx in top_indices:
        print(f'idx: {idx}')
        print(f'wiki: {wiki_list[idx][:100]}')
        print(f'correct context: {correct_contexts[i][:100]}')
        if correct_contexts[idx] == wiki_list[idx]:  # 정답이 포함되어 있는지 확인
            correct = True
            correct_count += 1  # 정답이 포함되면 카운트 증가
            break
    
    
    
    
    break

Processing query 1/1:   0%|          | 0/2 [00:00<?, ?it/s]

start idx: 0
batch size: 5
wiki list len: 10


Processing query 1/1:  50%|█████     | 1/2 [00:05<00:05,  5.44s/it]

(5,)
start idx: 5
batch size: 5
wiki list len: 10


Processing query 1/1: 100%|██████████| 2/2 [00:09<00:00,  4.87s/it]

(5,)
[array([70.44482, 73.37528, 70.29869, 73.86983, 63.21244], dtype=float32), array([64.66734, 62.94172, 72.91665, 69.45655, 69.65785], dtype=float32)]
[70.44482 73.37528 70.29869 73.86983 63.21244 64.66734 62.94172 72.91665
 69.45655 69.65785]
[3 1 7 0 2]
idx: 3
wiki: 신라 지증왕 4년(503년)에 만들어진 것으로 추정되며, 1989년에 발견되었다. 발견 당시 신라시대 비석 중 현존하는 최고(最古)의 비석이었으나, 2009년 9월에 501년 혹은
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 1
wiki: 원래 베트남 지역을 통치하던 쩐 왕조는 대대로 명나라에 공물을 바치는 속국이었다. 하지만 1400년에 쩐 왕조의 장군 호꾸이리가 반란을 일으켜 쩐 왕가를 대거 학살한 다음 제위에 
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 7
wiki: 1600년(선조 33년) 의인왕후 박씨가 승하하자 왕비릉인 유릉(裕陵)의 터로 정해진 곳이다.

1608년(선조 41년) 선조가 승하하고 광해군이 즉위하면서 능을 건원릉의 서편에 
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 0
wiki: 경주시 황남동 미추왕릉 지구에 있는 삼국시대 신라 무덤인 